本コンペは日次の野菜卸売価格の予測タスクですが、ここでは簡単のため、月次の予測タスクに読み替えてみます（日毎に予測値は変えない）<br>
本notebookでは前処理を行い、次のnotebookでは学習・推論を行います

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# コンペデータを格納しているパスを指定ください
PATH = '/content/drive/MyDrive/cpt-yasai-2023au/'

In [3]:
%cd $PATH

/content/drive/MyDrive/cpt-yasai-2023au


In [4]:
import pandas as pd
import numpy as np
from pathlib import Path

In [5]:
INPUT_DIR = Path("data/input/")

In [6]:
train_df = pd.read_csv(INPUT_DIR/'train.csv')
print(train_df.shape)
train_df

(69104, 7)


,kind,date,amount,mode_price,area,year,weekno
0,かぶ,20051107,39319,158.0,千葉,2005,45
1,かぶ,20051108,35433,158.0,千葉,2005,45
2,かぶ,20051110,40208,126.0,千葉,2005,45
3,かぶ,20051111,32032,137.0,千葉,2005,45
4,かぶ,20051112,38288,137.0,千葉,2005,45
...,...,...,...,...,...,...,...
69099,しめじ,20170526,17231,48.5,新潟_長野,2017,21
69100,しめじ,20170527,17105,48.5,新潟_長野,2017,21
69101,しめじ,20170529,12406,48.5,新潟_長野,2017,22
69102,しめじ,20170530,12320,48.5,新潟_長野,2017,22


In [7]:
test_df = pd.read_csv(INPUT_DIR/'test.csv')
print(test_df.shape)
test_df

(220, 5)


,kind,date,area,year,weekno
0,にんじん,20230701,北海道_各地_青森,2023,26
1,にんじん,20230703,北海道_各地_青森,2023,27
2,にんじん,20230704,北海道_各地_青森,2023,27
3,にんじん,20230706,北海道_各地_青森,2023,27
4,にんじん,20230707,北海道_各地_青森,2023,27
...,...,...,...,...,...
215,じゃがいも,20230725,各地_茨城_静岡,2023,30
216,じゃがいも,20230727,各地_茨城_静岡,2023,30
217,じゃがいも,20230728,各地_茨城_静岡,2023,30
218,じゃがいも,20230729,各地_茨城_静岡,2023,30


In [8]:
set(train_df.kind.unique()) >= set(test_df.kind.unique())

True

In [9]:
train_df.describe()

,date,amount,mode_price,year,weekno
count,6.910400e+04,69104.000000,69104.000000,69104.000000,69104.000000
mean,2.014039e+07,83800.419136,1344.053383,2013.972954,26.697514
std,5.057104e+04,88530.299942,1489.217123,5.059272,15.577581
min,2.005111e+07,1.000000,18.500000,2005.000000,1.000000
25%,2.010021e+07,18654.000000,167.500000,2010.000000,13.000000
50%,2.014052e+07,48403.500000,1188.000000,2014.000000,25.000000
75%,2.018101e+07,121612.500000,1836.000000,2018.000000,41.000000
max,2.023063e+07,954540.000000,25920.000000,2023.000000,53.000000


In [10]:
test_df.describe()

,date,year,weekno
count,2.200000e+02,220.0,220.00000
mean,2.023072e+07,2023.0,28.50000
std,8.942415e+00,0.0,1.30856
min,2.023070e+07,2023.0,26.00000
25%,2.023071e+07,2023.0,27.00000
50%,2.023072e+07,2023.0,28.50000
75%,2.023072e+07,2023.0,30.00000
max,2.023073e+07,2023.0,31.00000


In [11]:
train_df.groupby('date')[['date']].agg('count')

,date
date,
20051107,16
20051108,15
20051110,17
20051111,16
20051112,17
...,...
20230624,13
20230626,13
20230627,13


In [12]:
test_df.groupby('date')[['date']].agg('count')

,date
date,
20230701,10
20230703,10
20230704,10
20230706,10
20230707,10
20230708,10
20230710,10
20230711,10
20230713,10


In [13]:
sub_df = pd.read_csv(INPUT_DIR/'sample_submission.csv')
sub_df.describe()

,date,mode_price
count,2.200000e+02,220.0
mean,2.023072e+07,0.0
std,8.942415e+00,0.0
min,2.023070e+07,0.0
25%,2.023071e+07,0.0
50%,2.023072e+07,0.0
75%,2.023072e+07,0.0
max,2.023073e+07,0.0


In [20]:
wea_df = pd.read_csv(INPUT_DIR/'weather.csv')
print(wea_df.shape)
wea_df

(215200, 10)


,date,mean_temp,max_temp,max_temp_time,min_temp,min_temp_time,sum_rain,sun_time,mean_humid,area
0,20041106,13.1,20.1,2004/11/6 12:50,8.1,2004/11/6 05:31,5.0,1.6,77.0,青森
1,20041107,9.6,12.5,2004/11/7 13:40,5.4,2004/11/7 22:17,0.0,4.1,63.0,青森
2,20041108,9.0,15.5,2004/11/8 12:51,3.2,2004/11/8 06:28,0.0,8.9,72.0,青森
3,20041109,12.2,17.7,2004/11/9 14:40,5.7,2004/11/9 02:49,1.5,1.2,85.0,青森
4,20041110,11.6,16.4,2004/11/10 14:42,6.9,2004/11/10 06:23,0.0,7.5,89.0,青森
...,...,...,...,...,...,...,...,...,...,...
215195,20230327,13.4,16.9,2023/3/27 12:44,11.2,2023/3/28 0:00,0.0,2.3,55.0,東京
215196,20230328,10.2,13.4,2023/3/28 15:32,7.5,2023/3/28 10:20,3.0,1.7,80.0,東京
215197,20230329,12.6,18.6,2023/3/29 12:01,8.3,2023/3/29 03:19,15.5,3.7,72.0,東京
215198,20230330,14.1,19.7,2023/3/30 14:22,10.1,2023/3/30 03:36,0.5,5.8,76.0,東京


In [21]:
wea_df.describe()

,date,mean_temp,max_temp,min_temp,sum_rain,sun_time,mean_humid
count,2.152000e+05,214833.000000,214833.000000,214832.000000,214820.000000,214768.000000,214796.000000
mean,2.013621e+07,15.437010,19.959887,11.604558,4.407758,5.434972,69.242383
std,5.319825e+04,8.824639,8.989385,9.202588,13.512384,4.049137,13.198264
min,2.004111e+07,-16.000000,-9.100000,-24.500000,0.000000,0.000000,17.000000
25%,2.009061e+07,8.100000,12.800000,3.900000,0.000000,1.300000,60.000000
50%,2.014012e+07,16.000000,20.700000,11.800000,0.000000,5.600000,70.000000
75%,2.018083e+07,22.800000,27.300000,19.500000,2.000000,9.000000,79.000000
max,2.023033e+07,33.900000,41.100000,30.500000,429.500000,14.600000,100.000000


In [22]:
wea_df.groupby('area')[['area']].agg('count')

,area
area,
仙台,6725
佐賀,6725
前橋,6725
千葉,6725
名古屋,6725
和歌山,6725
宇都宮,6725
宮崎,6725
山形,6725


まず、天候データを加工します。月毎の基本統計量をとります

In [23]:
wea_df['year'] = wea_df['date'].apply(lambda x: int(str(x)[:4]))
wea_df['month'] = wea_df['date'].apply(lambda x: int(str(x)[4:6]))

In [24]:
agg_cols = ['mean_temp', 'max_temp', 'min_temp', 'sum_rain', 'sun_time', 'mean_humid']
gb_df = wea_df.groupby(['area', 'year', 'month'])[agg_cols].agg(['mean','max','min']).reset_index()

new_cols = []
for col1, col2 in gb_df.columns:
    if col2:
        new_cols.append(col2+'_'+col1)
    else:
        new_cols.append(col1)

gb_df.columns = new_cols
gb_df

,area,year,month,mean_mean_temp,max_mean_temp,min_mean_temp,mean_max_temp,max_max_temp,min_max_temp,mean_min_temp,...,min_min_temp,mean_sum_rain,max_sum_rain,min_sum_rain,mean_sun_time,max_sun_time,min_sun_time,mean_mean_humid,max_mean_humid,min_mean_humid
0,仙台,2004,11,11.256000,16.4,4.4,15.196000,19.3,7.5,7.152000,...,1.1,1.340000,20.0,0.0,4.436000,9.4,0.0,68.320000,93.0,45.0
1,仙台,2004,12,5.422581,13.3,-1.2,9.212903,21.6,0.4,1.825806,...,-4.4,2.048387,42.0,0.0,4.441935,8.9,0.0,64.225806,86.0,46.0
2,仙台,2005,1,1.590323,5.4,-2.0,5.077419,11.1,0.6,-1.809677,...,-5.6,2.000000,44.5,0.0,3.938710,9.4,0.0,69.709677,92.0,51.0
3,仙台,2005,2,0.828571,3.0,-1.3,4.335714,8.6,0.3,-2.221429,...,-4.7,2.000000,25.0,0.0,4.392857,9.0,0.0,68.750000,89.0,48.0
4,仙台,2005,3,4.061290,8.2,-1.2,8.332258,13.5,0.1,0.225806,...,-4.6,1.903226,28.0,0.0,5.522581,10.8,0.0,63.000000,89.0,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7067,鹿児島,2022,11,18.477419,21.9,14.8,22.916129,26.4,19.1,14.577419,...,9.8,5.483871,100.5,0.0,4.648387,10.0,0.0,72.935484,91.0,55.0
7068,鹿児島,2022,12,9.400000,15.9,4.2,13.754839,21.3,6.3,5.567742,...,0.4,2.500000,17.0,0.0,4.819355,9.6,0.0,69.677419,89.0,54.0
7069,鹿児島,2023,1,9.035484,18.6,0.5,14.016129,20.4,5.5,4.958065,...,-3.0,2.838710,35.5,0.0,4.848387,9.9,0.0,69.677419,95.0,56.0
7070,鹿児島,2023,2,11.546429,15.8,8.5,16.164286,20.4,12.5,7.514286,...,3.3,5.535714,46.0,0.0,5.000000,11.0,0.0,67.928571,87.0,50.0


train.csv, test.csvには「各地」というエリアがあります。天候データ側ではこれを「全国」と読み替えることにして、全国平均を算出しておきます

In [25]:
agg_cols = [i for i in gb_df.columns if i not in ['year', 'month', 'area']]
tmp_df = gb_df.groupby(['year', 'month'])[agg_cols].agg(['mean']).reset_index()

new_cols = []
for col1, col2 in tmp_df.columns:
    new_cols.append(col1)

tmp_df.columns = new_cols
tmp_df['area'] = '全国'
tmp_df = tmp_df[gb_df.columns]
tmp_df

,area,year,month,mean_mean_temp,max_mean_temp,min_mean_temp,mean_max_temp,max_max_temp,min_max_temp,mean_min_temp,...,min_min_temp,mean_sum_rain,max_sum_rain,min_sum_rain,mean_sun_time,max_sun_time,min_sun_time,mean_mean_humid,max_mean_humid,min_mean_humid
0,全国,2004,11,13.046750,18.481250,8.406250,17.580375,22.950000,11.609375,9.046250,...,4.287500,3.456875,42.578125,0.0,5.446625,9.478125,0.00000,67.877500,87.96875,49.59375
1,全国,2004,12,8.011492,14.731250,1.662500,12.469456,21.259375,4.171875,4.086089,...,-1.271875,3.554940,54.796875,0.0,4.952319,9.059375,0.00000,65.214718,85.18750,48.43750
2,全国,2005,1,4.076210,7.975000,0.912500,8.123286,14.028125,3.909375,0.388609,...,-3.300000,2.198085,26.328125,0.0,4.467498,8.940625,0.00625,64.005040,84.96875,46.59375
3,全国,2005,2,4.199888,9.156250,0.143750,8.256920,14.600000,2.921875,0.549442,...,-3.390625,2.989955,25.125000,0.0,4.314951,9.321875,0.00000,63.796875,88.56250,46.25000
4,全国,2005,3,7.202218,12.068750,1.228125,11.891431,18.268750,4.409375,2.865927,...,-2.656250,3.122984,29.046875,0.0,5.416835,10.934375,0.00000,61.677419,87.09375,41.53125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,全国,2022,11,13.829738,17.834375,10.440625,18.690524,23.690625,13.750000,9.565423,...,5.500000,3.108367,33.046875,0.0,5.342137,9.837500,0.00000,72.862500,91.56250,57.21875
217,全国,2022,12,6.109073,10.325000,1.575000,10.343548,15.931250,4.246875,2.422379,...,-1.756250,2.330645,20.562500,0.0,4.592036,8.753125,0.00000,68.473790,89.25000,48.15625
218,全国,2023,1,4.617944,11.371875,-2.000000,9.051512,15.850000,1.378125,0.701512,...,-5.281250,1.591734,18.765625,0.0,5.072379,9.359375,0.00625,66.551277,91.15625,49.25000
219,全国,2023,2,5.962723,11.293750,1.809375,10.541183,17.500000,4.534375,1.800000,...,-2.515625,1.977121,22.265625,0.0,5.360379,10.612500,0.00000,65.246652,88.87500,46.06250


都道府県別と全国平均の値を結合します

In [26]:
wea_df = pd.concat([gb_df, tmp_df])

卸売データと天候データを結合するには、各々でエリアの名称が異なるため、エリアの対応をとる必要があります<br>
エリアの変換を行うmapを作成します

In [27]:
all_df = pd.concat([train_df, test_df])
all_df

,kind,date,amount,mode_price,area,year,weekno
0,かぶ,20051107,39319.0,158.0,千葉,2005,45
1,かぶ,20051108,35433.0,158.0,千葉,2005,45
2,かぶ,20051110,40208.0,126.0,千葉,2005,45
3,かぶ,20051111,32032.0,137.0,千葉,2005,45
4,かぶ,20051112,38288.0,137.0,千葉,2005,45
...,...,...,...,...,...,...,...
215,じゃがいも,20230725,NaN,NaN,各地_茨城_静岡,2023,30
216,じゃがいも,20230727,NaN,NaN,各地_茨城_静岡,2023,30
217,じゃがいも,20230728,NaN,NaN,各地_茨城_静岡,2023,30
218,じゃがいも,20230729,NaN,NaN,各地_茨城_静岡,2023,30


卸売データのエリアを列挙します

In [28]:
area_pairs = all_df['area'].unique()

yasai_areas = set()

for area_pair in area_pairs:
    areas = area_pair.split('_')
    yasai_areas |= set(areas)

yasai_areas

{'アメリカ',
 'カナダ',
 'トンガ',
 'ニュージーランド',
 'メキシコ',
 '中国',
 '佐賀',
 '兵庫',
 '北海道',
 '千葉',
 '各地',
 '和歌山',
 '埼玉',
 '宮城',
 '宮崎',
 '岩手',
 '徳島',
 '愛媛',
 '愛知',
 '新潟',
 '東京',
 '栃木',
 '沖縄',
 '熊本',
 '神奈川',
 '福岡',
 '福島',
 '秋田',
 '群馬',
 '茨城',
 '長崎',
 '長野',
 '青森',
 '静岡',
 '香川',
 '高知',
 '鹿児島'}

天候データのエリアを列挙します

In [29]:
wea_areas = wea_df['area'].unique()
wea_areas

array(['仙台', '佐賀', '前橋', '千葉', '名古屋', '和歌山', '宇都宮', '宮崎', '山形', '帯広',
       '徳島', '新潟', '東京', '松山', '横浜', '水戸', '浜松', '熊本', '熊谷', '甲府', '盛岡',
       '神戸', '福岡', '福島', '秋田', '那覇', '長崎', '長野', '青森', '高松', '高知', '鹿児島',
       '全国'], dtype=object)

エリア変換のmapを作ります

In [30]:
area_map = {}

update_area_map = {
    '岩手':'盛岡','宮城':'仙台','静岡':'浜松','沖縄':'那覇','神奈川':'横浜','愛知':'名古屋','茨城':'水戸','北海道':'帯広','各地':'全国',
    '兵庫':'神戸','香川':'高松','埼玉':'熊谷','国内':'全国','山梨':'甲府','栃木':'宇都宮','群馬':'前橋','愛媛':'松山'
}

for yasai_area in yasai_areas:
    if yasai_area not in wea_areas and yasai_area not in update_area_map:
        area_map[yasai_area] = '全国' # 外国の天候は全国にしておく
    else:
        area_map[yasai_area] = yasai_area

area_map = {**area_map, **update_area_map}
area_map

{'北海道': '帯広',
 '宮城': '仙台',
 '各地': '全国',
 '埼玉': '熊谷',
 'ニュージーランド': '全国',
 '愛知': '名古屋',
 '岩手': '盛岡',
 '高知': '高知',
 '栃木': '宇都宮',
 'トンガ': '全国',
 '福島': '福島',
 '和歌山': '和歌山',
 '長野': '長野',
 '宮崎': '宮崎',
 '長崎': '長崎',
 '秋田': '秋田',
 '福岡': '福岡',
 '東京': '東京',
 '茨城': '水戸',
 '新潟': '新潟',
 '沖縄': '那覇',
 '兵庫': '神戸',
 '青森': '青森',
 '鹿児島': '鹿児島',
 '中国': '全国',
 '静岡': '浜松',
 '香川': '高松',
 '千葉': '千葉',
 '神奈川': '横浜',
 '徳島': '徳島',
 'カナダ': '全国',
 'アメリカ': '全国',
 '群馬': '前橋',
 '愛媛': '松山',
 '佐賀': '佐賀',
 '熊本': '熊本',
 'メキシコ': '全国',
 '国内': '全国',
 '山梨': '甲府'}

卸売データのエリアを天候データのエリアに変換します

In [31]:
all_df['area'] = all_df['area'].apply(lambda x: '_'.join([area_map[i] for i in x.split('_')]))
all_df

,kind,date,amount,mode_price,area,year,weekno
0,かぶ,20051107,39319.0,158.0,千葉,2005,45
1,かぶ,20051108,35433.0,158.0,千葉,2005,45
2,かぶ,20051110,40208.0,126.0,千葉,2005,45
3,かぶ,20051111,32032.0,137.0,千葉,2005,45
4,かぶ,20051112,38288.0,137.0,千葉,2005,45
...,...,...,...,...,...,...,...
215,じゃがいも,20230725,NaN,NaN,全国_水戸_浜松,2023,30
216,じゃがいも,20230727,NaN,NaN,全国_水戸_浜松,2023,30
217,じゃがいも,20230728,NaN,NaN,全国_水戸_浜松,2023,30
218,じゃがいも,20230729,NaN,NaN,全国_水戸_浜松,2023,30


In [32]:
test_df = all_df.iloc[train_df.shape[0]:]
train_df = all_df.iloc[:train_df.shape[0]]

ここまで加工したデータを出力します（出力せず、このまま学習・推論を行っても構いません）

In [34]:
OUTPUT_DIR = Path("data/input/")
OUTPUT_DIR.mkdir(exist_ok=True)

In [35]:
test_df.to_csv(OUTPUT_DIR/'tutorial_preprocessed_test.csv', index=False)
train_df.to_csv(OUTPUT_DIR/'tutorial_preprocessed_train.csv', index=False)

一方で天候データについても、卸売データと単純にmergeするには加工が必要です<br>「千葉_全国_青森」のように複数エリアに跨っている場合の各統計量について、各エリアの平均を取ることにします

In [36]:
area_pairs = all_df['area'].unique()

In [37]:
target_cols = [i for i in wea_df.columns if i != 'area']

In [38]:
area_pair_dfs = []

for area_pair in area_pairs:
    areas = area_pair.split('_')
    if len(areas) > 0:
        area = areas[0]
        base_tmp_df = wea_df[wea_df['area'] == area]
        base_tmp_df = base_tmp_df[target_cols].reset_index(drop=True)
        for area in areas[1:]:
            tmp_df = wea_df[wea_df['area'] == area]
            tmp_df = tmp_df[target_cols].reset_index(drop=True)
            base_tmp_df = base_tmp_df.add(tmp_df)
        base_tmp_df /= len(areas)
        base_tmp_df['area'] = area_pair
        area_pair_dfs.append(base_tmp_df)

In [39]:
area_pair_df = pd.concat(area_pair_dfs)
print(area_pair_df.shape)
area_pair_df.head()

(43537, 21)


,year,month,mean_mean_temp,max_mean_temp,min_mean_temp,mean_max_temp,max_max_temp,min_max_temp,mean_min_temp,max_min_temp,...,mean_sum_rain,max_sum_rain,min_sum_rain,mean_sun_time,max_sun_time,min_sun_time,mean_mean_humid,max_mean_humid,min_mean_humid,area
0,2004.0,11.0,14.596000,19.3,9.6,18.340000,22.2,12.4,11.228000,17.4,...,5.840000,87.0,0.0,5.084000,9.5,0.0,71.760000,93.0,43.0,千葉
1,2004.0,12.0,9.641935,19.0,2.2,13.564516,24.3,3.8,6.138710,11.5,...,2.435484,26.0,0.0,5.509677,9.4,0.0,58.612903,86.0,37.0,千葉
2,2005.0,1.0,6.277419,13.1,3.1,10.461290,17.2,5.2,2.522581,6.4,...,3.338710,66.5,0.0,6.612903,9.8,0.0,54.838710,89.0,32.0,千葉
3,2005.0,2.0,5.960714,12.0,3.3,9.864286,15.4,5.1,2.060714,4.6,...,1.910714,11.5,0.0,5.225000,9.7,0.0,55.714286,93.0,31.0,千葉
4,2005.0,3.0,8.800000,13.7,1.6,13.122581,18.4,3.6,4.596774,10.0,...,2.612903,22.5,0.0,5.529032,11.3,0.0,59.129032,91.0,31.0,千葉


ここまで加工したデータを出力します（出力せず、このまま学習・推論を行っても構いません）

In [40]:
area_pair_df.to_csv(OUTPUT_DIR/'tutorial_preprocessed_weather.csv', index=False)